# Import Library

In [1]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt

C:\Users\ahska\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


# Upload File

In [2]:
# Load data
redfin_data = pd.read_csv('C:/Users/ahska/Documents/Ahska/UCL/3 - Final Project (Dissertation)/Dissertation 2/Main Data/Redfin Dataset (Final 2)/redfin_seattle_dataset_house.csv')
poi_data = pd.read_csv('C:/Users/ahska/Documents/Ahska/UCL/3 - Final Project (Dissertation)/Dissertation 2/POI Data/poi_database_final_cleaned.csv')

C:\Users\ahska\AppData\Local\Temp\ipykernel_31428\3962044794.py:3: DtypeWarning: Columns (3,9,10,11,12,15) have mixed types. Specify dtype option on import or set low_memory=False.
  poi_data = pd.read_csv('C:/Users/ahska/Documents/Ahska/UCL/3 - Final Project (Dissertation)/Dissertation 2/POI Data/poi_database_final_cleaned.csv')


# Haversine Formula

In [3]:
import pandas as pd
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # Convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of Earth in kilometers
    return c * r

In [4]:
haversine(-122.34784, 47.745233, -122.3451336, 47.7494139)

0.507023869550569

# Density Formula

In [5]:
def preliminary_filter(lat, lon, D=1.5):
    """Create a bounding box to pre-filter POIs within approximate range."""
    delta_lat = D / 111  # Roughly 1.5 km in latitude
    delta_lon = D / (111 * cos(radians(lat)))  # Adjust longitude delta by cosine of latitude
    return (poi_data['latitude'].between(lat - delta_lat, lat + delta_lat) &
            poi_data['longitude'].between(lon - delta_lon, lon + delta_lon))

def accessibility_measure(property_row, index, total, D=1.5):
    """Calculate the accessibility measure for each POI type based on linear decay function."""
    lat, lon = property_row['lat'], property_row['long']
    accessibility_scores = {}
    nearby_pois = poi_data[preliminary_filter(lat, lon, D)]
    for poi_type in poi_types:
        type_pois = nearby_pois[nearby_pois['sub_2_primarytype'].str.contains(poi_type)]
        score = 0
        for _, poi in type_pois.iterrows():
            poi_lat, poi_lon = poi['latitude'], poi['longitude']
            distance = haversine(lon, lat, poi_lon, poi_lat)
            if distance < D:
                score += (1 - distance / D)  # Apply decay function
        accessibility_scores[f"{poi_type}_accessibility"] = score
    
    # Progress reporting
    if index % (total // 10) == 0:
        print(f"Processing... {index / total * 100:.0f}% completed")
    return pd.Series(accessibility_scores)

In [6]:
# # Apply the function across all properties
# redfin_data[poi_types] = redfin_data.apply(calculate_poi_density, axis=1)

# # Save the results or display them
# print(redfin_data.head())

# Defining Important POI Types

In [7]:
# POI types of interest
poi_types = [
    'gas_station', 'parking',
    'airport', 'bus_station', 'ferry_terminal', 'light_rail_station', 'train_station', 'transit_station', 'transportation_others',
    'library', 'preschool', 'school', 'university',
    'amusement_center', 'community_center', 'entertainment',
    'park',
    'bar', 'cafe_and_bakery', 'food_others', 'restaurant',
    'health_store', 'hospital',
    'places_of_worship',
    'safety',
    'stores_essentials', 'stores_others'
]


# Running the Formula

In [8]:
# Calculate accessibility measure for each property
total_rows = len(redfin_data)
redfin_data = redfin_data.join(redfin_data.apply(lambda row: accessibility_measure(row, row.name, total_rows, D=1.5), axis=1))


Processing... 0% completed
Processing... 10% completed
Processing... 20% completed
Processing... 30% completed
Processing... 40% completed
Processing... 50% completed
Processing... 60% completed
Processing... 70% completed
Processing... 80% completed
Processing... 90% completed
Processing... 100% completed


In [9]:
# Save the updated DataFrame to a new CSV file
redfin_data.to_csv('updated_redfin_data_with_accessibility_scores_v3.csv', index=False)

In [10]:
# Display some of the updated DataFrame to check results
print("Processing complete. Head of the updated DataFrame:")
print(redfin_data.head())

Processing complete. Head of the updated DataFrame:
     SALE TYPE        SOLD DATE              PROPERTY TYPE           ADDRESS  \
0    PAST SALE  November-3-2023  Single Family Residential    443 11th Ave W   
1    PAST SALE     June-30-2023  Single Family Residential  2817 88th Ave NE   
2    PAST SALE     July-21-2023  Single Family Residential  1008 88th Ave NE   
3    PAST SALE  January-27-2023  Single Family Residential   1520 90th Pl NE   
4  MLS Listing              NaN  Single Family Residential   8830 NE 28th St   

          CITY STATE OR PROVINCE  ZIP OR POSTAL CODE  \
0     Kirkland                WA               98003   
1  Hunts Point                WA               98004   
2     Bellevue                WA               98004   
3   Clyde Hill                WA               98004   
4   Clyde Hill                WA               98004   

                            NEIGHBORHOOD     PRICE  BEDS  ...  \
0                            Federal Way   2700000     2  ...   


In [11]:
# Step 1: Identify all columns that represent accessibility measures
accessibility_columns = [col for col in redfin_data.columns if '_accessibility' in col]

# Step 2: Compute the average accessibility measure for each POI type
average_accessibility = redfin_data[accessibility_columns].mean()

# Step 3: Sort the averages in descending order
sorted_accessibility = average_accessibility.sort_values(ascending=False)

# Display the sorted averages
print("Average Accessibility Measures Sorted:")
print(sorted_accessibility)

Average Accessibility Measures Sorted:
restaurant_accessibility               26.027815
park_accessibility                     25.174492
stores_others_accessibility            23.638285
bus_station_accessibility              21.232271
parking_accessibility                  13.715487
transit_station_accessibility          13.200188
cafe_and_bakery_accessibility           9.039253
school_accessibility                    8.688018
food_others_accessibility               5.808909
bar_accessibility                       5.151770
places_of_worship_accessibility         5.111753
stores_essentials_accessibility         4.854828
health_store_accessibility              3.625721
entertainment_accessibility             2.384405
preschool_accessibility                 1.836418
university_accessibility                1.767358
hospital_accessibility                  1.742096
library_accessibility                   1.519190
gas_station_accessibility               1.193679
community_center_accessibility